# Ficha de trabalho 4

1- Considera o dataset cachexia. Lê os dados usando a função read_csv do pandas (dados e metadados).

In [1]:
from pandas import read_csv
data_metab = read_csv("data_cachexia.csv", sep=',', index_col = 0)
meta_metab = read_csv("meta_cachexia.csv", sep = ",", index_col = 0)
print(data_metab.shape)
print(meta_metab.shape)

(77, 63)
(77, 1)


2 - Aplica a transformação necessária para que os dados seja escalonados para ter média 0 e desvio padrão 1. Verifica que as médias de todas as colunas é aproximadamente zero.

In [2]:
from sklearn import preprocessing

input_data = data_metab.values
output_data = meta_metab.values[:,0]

input_sc = preprocessing.scale(input_data) #media 0 e desvio padrão=1

print("Media global: ", input_sc.mean())
print("Desvio padrao global: ", input_sc.std())


Media global:  -1.8675365658506033e-17
Desvio padrao global:  1.0


In [3]:
((input_sc.mean(axis=0) < 0.000001) & (input_sc.mean(axis=0) > -0.000001)).all()

# poder-se-ia verificar a média de cada um dos atributos 
#for i in range(63):
#    print(input_sc[:,i].mean())


True

In [4]:
((input_sc.std(axis=0) < 1.000001) & (input_sc.std(axis=0) > 0.999999)).all()

True

3 - Cria dois conjuntos de dados treino e teste para serem usados na criação e validação de modelos de aprendizagem máquina. Considera 30% das amostras para formar o conjunto de teste. 

In [5]:
import numpy as np

numtst = int(input_sc.shape[0] *0.3)
print("Numero de exemplos para teste: ", numtst)

indices = np.random.permutation(len(input_data))
indices

train_in = input_sc[indices[:-numtst]]
train_out = output_data[indices[:-numtst]]

test_in  = input_sc[indices[-numtst:]]
test_out = output_data[indices[-numtst:]]

Numero de exemplos para teste:  23


In [6]:
## verificando as dimensoes
print(train_in.shape)
print(train_out.shape)

print(test_in.shape)
print(test_out.shape)

(54, 63)
(54,)
(23, 63)
(23,)


4 - Constroi diversos tipos de modelos de classificação treinando e testando com os conjuntos criados anteriormente. Calcula, para cada um dos modelos no test set, as métricas de erro PECC e F1-score, e calcula a matriz de confusão. Considere como classe positiva a classe "cachexic"

In [7]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(train_in, train_out)
prev = knn.predict(test_in)

print(knn.score(test_in, test_out))

0.6086956521739131


In [8]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

print("PECC: ", accuracy_score(test_out, prev) )
print("F1-score: ", f1_score(test_out, prev, pos_label = "cachexic") )

confusion_matrix(test_out, prev)

PECC:  0.6086956521739131
F1-score:  0.64


array([[8, 6],
       [3, 6]])

In [9]:
tn, fp, fn, tp = confusion_matrix(test_out, prev).ravel()
print("VP: ", tp)
print("VN: ", tn)
print("FP: ", fp)
print("FN: ", fn)

VP:  6
VN:  8
FP:  6
FN:  3


In [10]:
from sklearn import tree

tree_model = tree.DecisionTreeClassifier()
tree_model = tree_model.fit(train_in, train_out)
prev_tree = tree_model.predict(test_in)

print("PECC: ", accuracy_score(test_out, prev_tree) )
print("F1-score: ", f1_score(test_out, prev_tree, pos_label = "cachexic") )
confusion_matrix(test_out, prev_tree)

PECC:  0.782608695652174
F1-score:  0.8275862068965518


array([[12,  2],
       [ 3,  6]])

In [11]:
from sklearn.naive_bayes import GaussianNB

gnb_model = GaussianNB()
gnb_model = gnb_model.fit(train_in, train_out)

prev_gnb = gnb_model.predict(test_in)

print("PECC: ", accuracy_score(test_out, prev_gnb) )
print("F1-score: ", f1_score(test_out, prev_gnb, pos_label = "cachexic") )
confusion_matrix(test_out, prev_gnb)

PECC:  0.6086956521739131
F1-score:  0.6086956521739131


array([[7, 7],
       [2, 7]])

5 - Usando apenas o melhor modelo do exercício anterior, compare o seu desempenho considerando dados com transformação logaritmica e posteriormente standardizados

In [12]:
input_log = np.log2(input_data)

In [13]:
input_log_sc = preprocessing.scale(input_log)

train_in_log = input_log_sc[indices[:-numtst]]
test_in_log  = input_log_sc[indices[-numtst:]]

In [14]:
tree_model_log = tree.DecisionTreeClassifier()
tree_model_log = tree_model_log.fit(train_in_log, train_out)
prev_tree_log = tree_model_log.predict(test_in_log)

print("PECC: ", accuracy_score(test_out, prev_tree_log) )
print("F1-score: ", f1_score(test_out, prev_tree_log, pos_label = "cachexic") )
confusion_matrix(test_out, prev_tree_log)

PECC:  0.6521739130434783
F1-score:  0.7142857142857143


array([[10,  4],
       [ 4,  5]])